In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.stats import linregress
from datetime import datetime, timedelta

# === Config ===
input_path = "../data/BTCUSDT_1min.csv"
regime_dir = "data_experiment/regime_fragments"
output_dir = "data_experiment/backtest_fragments"
csv_path = os.path.join(output_dir, "best_fragments_combined.csv")
os.makedirs(regime_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

fragment_length = 720  # 12 hours
step_size = 120        # 1 hour step

# === Step 1: Load OHLCV ===
df_btc = pd.read_csv(input_path, parse_dates=["timestamp"])
df_btc.set_index("timestamp", inplace=True)
df_close = df_btc[["close"]].copy()

# === Step 2: Extract sliding windows and classify regimes ===
window_centers, regimes, vols, slopes = [], [], [], []

for i in range(0, len(df_close) - fragment_length, step_size):
    window_df = df_close.iloc[i:i + fragment_length]
    x = np.arange(len(window_df))
    slope = linregress(x, window_df["close"]).slope
    vol = window_df["close"].pct_change().std()
    center_time = window_df.index[len(window_df) // 2]

    slopes.append(slope)
    vols.append(vol)
    window_centers.append(center_time)

# === Step 3: Define thresholds ===
vol_thresh = np.median(vols)
slope_thresh = np.percentile(np.abs(slopes), 10)

# === Step 4: Assign regimes ===
for slope, vol in zip(slopes, vols):
    if abs(slope) < slope_thresh:
        regimes.append("Sideways")
    elif slope > 0 and vol > vol_thresh:
        regimes.append("HighVol_UpTrend")
    elif slope > 0:
        regimes.append("LowVol_UpTrend")
    elif slope < 0 and vol > vol_thresh:
        regimes.append("HighVol_DownTrend")
    else:
        regimes.append("LowVol_DownTrend")

# === Step 5: Slice fragments and save by regime ===
fragments = []
for i, center_time in enumerate(window_centers):
    label = regimes[i]
    start_idx = i * step_size
    end_idx = start_idx + fragment_length
    df_fragment = df_btc.iloc[start_idx:end_idx].copy()
    df_fragment["regime_name"] = label
    fragments.append(df_fragment)

df_all_fragments = pd.concat(fragments)
df_all_fragments.reset_index(inplace=True)

# === Step 6: Save regime-specific fragment CSVs ===
for label in df_all_fragments["regime_name"].unique():
    df_sub = df_all_fragments[df_all_fragments["regime_name"] == label].copy()
    df_sub.to_csv(os.path.join(regime_dir, f"fragment_data_{label}.csv"), index=False)

print("\n[✓] Saved regime fragments:")
print(df_all_fragments["regime_name"].value_counts())

# === Step 7: Load and tag fragments ===
fragment_dfs = []
for file in os.listdir(regime_dir):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(regime_dir, file), parse_dates=["timestamp"])
        regime = file.replace("fragment_data_", "").replace(".csv", "")
        df["regime_name"] = regime
        df["fragment_id"] = np.repeat(np.arange(len(df) // fragment_length), fragment_length)
        fragment_dfs.append(df)

if not fragment_dfs:
    raise ValueError("No valid CSV fragments found in regime folder.")

df_all = pd.concat(fragment_dfs)

# === Step 8: Compute slope & volatility per fragment ===
features = []
for (regime, fid), group in df_all.groupby(["regime_name", "fragment_id"]):
    if len(group) < fragment_length:
        continue
    x = np.arange(len(group))
    y = group["close"].values
    slope = linregress(x, y).slope
    volatility = group["close"].pct_change().std()

    features.append({
        "regime_name": regime,
        "fragment_id": fid,
        "slope": slope,
        "slope_abs": abs(slope),
        "volatility": volatility
    })

df_feat = pd.DataFrame(features)

# === Step 9: Normalize and Score ===
df_feat["vol_norm"] = df_feat.groupby("regime_name")["volatility"].transform(
    lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0
)
df_feat["slope_norm"] = df_feat.groupby("regime_name")["slope_abs"].transform(
    lambda x: (x - x.min()) / (x.max() - x.min()) if x.max() != x.min() else 0
)

def score(row):
    if row["regime_name"] == "Sideways":
        return (1 - row["slope_norm"]) * (1 - row["vol_norm"])
    elif "UpTrend" in row["regime_name"]:
        return row["slope_norm"] * (1 - row["vol_norm"]) if "LowVol" in row["regime_name"] else row["slope_norm"] * row["vol_norm"]
    elif "DownTrend" in row["regime_name"]:
        return row["slope_norm"] * (1 - row["vol_norm"]) if "LowVol" in row["regime_name"] else row["slope_norm"] * row["vol_norm"]
    else:
        return 0.0

df_feat["score"] = df_feat.apply(score, axis=1)

# === Step 10: Select and save best-scoring fragment per regime ===
best_fragments = df_feat.sort_values("score", ascending=False).groupby("regime_name").first().reset_index()

selected_dfs = []
for _, row in best_fragments.iterrows():
    regime = row["regime_name"]
    fid = row["fragment_id"]
    path = os.path.join(regime_dir, f"fragment_data_{regime}.csv")
    df = pd.read_csv(path, parse_dates=["timestamp"])
    df["fragment_id"] = np.repeat(np.arange(len(df) // fragment_length), fragment_length)
    df_best = df[df["fragment_id"] == fid].copy()
    selected_dfs.append(df_best)

df_final = pd.concat(selected_dfs).sort_values("timestamp")
output_path = os.path.join(output_dir, "best_fragments_combined.csv")
df_final.to_csv(output_path, index=False)

print("Saved best 12-hour fragment per regime → {output_path}")
print(" Regimes selected:", df_final["regime_name"].unique())

In [6]:
import pandas as pd
import plotly.graph_objects as go
import os

# === Load best regime fragments ===
df = pd.read_csv(csv_path, parse_dates=["timestamp"])
df.set_index("timestamp", inplace=True)

# === Sort and group ===
df = df.sort_index()
regimes = df["regime_name"].unique()
top_regimes = list(regimes[:5])  # Limit to 5 plots

for regime in top_regimes:
    df_plot = df[df["regime_name"] == regime].copy()

    # Clamp outlier highs/lows
    q1 = df_plot["close"].quantile(0.25)
    q3 = df_plot["close"].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 3 * iqr
    upper = q3 + 3 * iqr
    df_plot["high"] = df_plot["high"].clip(upper=upper)
    df_plot["low"] = df_plot["low"].clip(lower=lower)

    # === Plot ===
    fig = go.Figure()

    fig.add_trace(go.Candlestick(
        x=df_plot.index,
        open=df_plot["open"],
        high=df_plot["high"],
        low=df_plot["low"],
        close=df_plot["close"],
        name="Price",
        increasing_line_color='lime',
        decreasing_line_color='red'
    ))

    fig.update_layout(
        title=f"Best 12-Hour Fragment – {regime}",
        xaxis_title="Time",
        yaxis_title="Price",
        height=600,
        xaxis_rangeslider_visible=False,
        template="plotly_white",
        hovermode="x unified"
    )

    fig.update_xaxes(range=[df_plot.index.min(), df_plot.index.max()])
    fig.show()


In [7]:
# include both vol filter and confidence threshold

from signals.model_preparation import prepare_features_only
from train_model.signal_model import SignalModel
from train_model.vol_model import VolatilityModel
from logs.prediction_logger import PredictionLogger
from logs.signal_logger import SignalHistoryLogger

# === Load data ===
df_fragments = pd.read_csv(csv_path, parse_dates=["timestamp"])
df_fragments.set_index("timestamp", inplace=True)
df_fragments.sort_index(inplace=True)

# === Load models ===
signal_model = SignalModel().load()
vol_model = VolatilityModel().load()
vol_features = vol_model.selected_features
sig_features = signal_model.selected_features

# === Result Summary ===
summary = []

# === Loop over each regime ===
for regime in df_fragments["regime_name"].unique():
    print(f"\n=== Evaluating Regime: {regime} ===")
    df = df_fragments[df_fragments["regime_name"] == regime].copy()
    df = prepare_features_only(df).dropna()
    print(f"[{regime}] Rows after feature prep: {len(df)}")

    if df.empty or len(df) < 10:
        print(f"[{regime}] Skipped due to insufficient rows after preprocessing.")
        continue

    signal_logger = SignalHistoryLogger()
    prediction_logger = PredictionLogger(autosave=False)

    for i in range(5, len(df) - 1):
        window = df.iloc[:i + 1]
        now = window.index[-2]

        vol_input = window.iloc[[-2]].reindex(columns=vol_features, fill_value=0.0)
        sig_input = window.iloc[[-2]].reindex(columns=sig_features, fill_value=0.0)

        try:
            vol_pred = vol_model.final_model.predict(vol_input)[0]
            if vol_pred != 1:
                # print(f"[{now}] {regime} → Skipped due to low volatility prediction")
                continue

            prob = signal_model.final_model.predict_proba(sig_input)[0][1]
            if prob >= 0.85:
                prediction = "UP"
            elif prob <= 0.15:
                prediction = "DOWN"
            else:
                # print(f"[{now}] {regime} → Skipped due to low confidence ({prob:.2%})")
                continue

            signal_type = "xgboost_bullish" if prediction == "UP" else "xgboost_bearish"
            conf_str = f"Conf={prob:.2%}" if prediction == "UP" else f"Conf={(1 - prob):.2%}"
            signal_logger.add_signal(signal_type, now, window["close"].iloc[-2], conf_str)

            prediction_logger.record_prediction(
                timestamp=now,
                prediction=prediction,
                close_now=window['close'].iloc[-1],
                close_prev=window['close'].iloc[-2],
                confidence=prob
            )

        except Exception as e:
            print(f"[{now}] {regime} → ERROR during prediction: {e}")
            continue

    df_result = prediction_logger.to_dataframe()
    num_preds = len(df_result)
    hit_rate = df_result["hit"].mean() if num_preds > 0 else 0.0

    print(f"[{regime}] Finished with {num_preds} predictions, hit rate: {hit_rate:.2%}")

    if num_preds > 0:
        summary.append({
            "regime": regime,
            "hit_rate": round(hit_rate * 100, 2),
            "num_predictions": num_preds
        })
    else:
        print(f"[{regime}] No confident predictions made.")

# === Final Summary ===
print("\n=== Backtest Summary ===")
if summary:
    summary_df = pd.DataFrame(summary)
    summary_df = summary_df.sort_values(by="hit_rate", ascending=False).reset_index(drop=True)
    print(summary_df)
else:
    print("No valid predictions were made across all regimes.")

[Load] Model and features loaded.
[Load] Last trained: 2025-07-04T04:02:04.560046
[VolLoad] Model loaded with 4 features.
[VolLoad] Last trained: 2025-07-04T04:01:13.302181

=== Evaluating Regime: HighVol_DownTrend ===
[HighVol_DownTrend] Rows after feature prep: 501
[HighVol_DownTrend] Finished with 68 predictions, hit rate: 39.71%

=== Evaluating Regime: HighVol_UpTrend ===
[HighVol_UpTrend] Rows after feature prep: 501
[HighVol_UpTrend] Finished with 42 predictions, hit rate: 40.48%

=== Evaluating Regime: Sideways ===
[Sideways] Rows after feature prep: 501
[Sideways] Finished with 0 predictions, hit rate: 0.00%
[Sideways] No confident predictions made.

=== Evaluating Regime: LowVol_DownTrend ===
[LowVol_DownTrend] Rows after feature prep: 501
[LowVol_DownTrend] Finished with 2 predictions, hit rate: 100.00%

=== Evaluating Regime: LowVol_UpTrend ===
[LowVol_UpTrend] Rows after feature prep: 501
[LowVol_UpTrend] Finished with 0 predictions, hit rate: 0.00%
[LowVol_UpTrend] No conf

In [8]:
# Remove confidence threshold

# === Result Summary ===
summary = []

# === Loop over each regime ===
for regime in df_fragments["regime_name"].unique():
    print(f"\n=== Evaluating Regime: {regime} ===")
    df = df_fragments[df_fragments["regime_name"] == regime].copy()
    df = prepare_features_only(df).dropna()
    print(f"[{regime}] Rows after feature prep: {len(df)}")

    if df.empty or len(df) < 10:
        print(f"[{regime}] Skipped due to insufficient rows after preprocessing.")
        continue

    signal_logger = SignalHistoryLogger()
    prediction_logger = PredictionLogger(autosave=False)

    for i in range(5, len(df) - 1):
        window = df.iloc[:i + 1]
        now = window.index[-2]

        vol_input = window.iloc[[-2]].reindex(columns=vol_features, fill_value=0.0)
        sig_input = window.iloc[[-2]].reindex(columns=sig_features, fill_value=0.0)

        try:
            vol_pred = vol_model.final_model.predict(vol_input)[0]
            if vol_pred != 1:
                # print(f"[{now}] {regime} → Skipped due to low volatility prediction")

                continue

            prob = signal_model.final_model.predict_proba(sig_input)[0][1]
            prediction = "UP" if prob >= 0.5 else "DOWN"

            signal_type = "xgboost_bullish" if prediction == "UP" else "xgboost_bearish"
            conf_str = f"Conf={prob:.2%}" if prediction == "UP" else f"Conf={(1 - prob):.2%}"
            signal_logger.add_signal(signal_type, now, window["close"].iloc[-2], conf_str)

            prediction_logger.record_prediction(
                timestamp=now,
                prediction=prediction,
                close_now=window['close'].iloc[-1],
                close_prev=window['close'].iloc[-2],
                confidence=prob
            )

        except Exception as e:
            print(f"[{now}] {regime} → ERROR during prediction: {e}")
            continue

    df_result = prediction_logger.to_dataframe()
    num_preds = len(df_result)
    hit_rate = df_result["hit"].mean() if num_preds > 0 else 0.0

    print(f"[{regime}] Finished with {num_preds} predictions, hit rate: {hit_rate:.2%}")

    if num_preds > 0:
        summary.append({
            "regime": regime,
            "hit_rate": round(hit_rate * 100, 2),
            "num_predictions": num_preds
        })
    else:
        print(f"[{regime}] No predictions made.")

# === Final Summary ===
print("\n=== Backtest Summary ===")
if summary:
    summary_df = pd.DataFrame(summary)
    summary_df = summary_df.sort_values(by="hit_rate", ascending=False).reset_index(drop=True)
    print(summary_df)
else:
    print("No valid predictions were made across all regimes.")


=== Evaluating Regime: HighVol_DownTrend ===
[HighVol_DownTrend] Rows after feature prep: 501
[HighVol_DownTrend] Finished with 495 predictions, hit rate: 44.24%

=== Evaluating Regime: HighVol_UpTrend ===
[HighVol_UpTrend] Rows after feature prep: 501
[HighVol_UpTrend] Finished with 495 predictions, hit rate: 43.43%

=== Evaluating Regime: Sideways ===
[Sideways] Rows after feature prep: 501
[Sideways] Finished with 0 predictions, hit rate: 0.00%
[Sideways] No predictions made.

=== Evaluating Regime: LowVol_DownTrend ===
[LowVol_DownTrend] Rows after feature prep: 501
[LowVol_DownTrend] Finished with 2 predictions, hit rate: 100.00%

=== Evaluating Regime: LowVol_UpTrend ===
[LowVol_UpTrend] Rows after feature prep: 501
[LowVol_UpTrend] Finished with 0 predictions, hit rate: 0.00%
[LowVol_UpTrend] No predictions made.

=== Backtest Summary ===
              regime  hit_rate  num_predictions
0   LowVol_DownTrend    100.00                2
1  HighVol_DownTrend     44.24              4

In [10]:
# Remove Vol filter

# === Result Summary ===
summary = []

# === Loop over each regime ===
for regime in df_fragments["regime_name"].unique():
    print(f"\n=== Evaluating Regime: {regime} ===")
    df = df_fragments[df_fragments["regime_name"] == regime].copy()
    df = prepare_features_only(df).dropna()
    print(f"[{regime}] Rows after feature prep: {len(df)}")

    if df.empty or len(df) < 10:
        print(f"[{regime}] Skipped due to insufficient rows after preprocessing.")
        continue

    signal_logger = SignalHistoryLogger()
    prediction_logger = PredictionLogger(autosave=False)

    for i in range(5, len(df) - 1):
        window = df.iloc[:i + 1]
        now = window.index[-2]

        # Skip vol filter — just compute inputs but don't condition on it
        vol_input = window.iloc[[-2]].reindex(columns=vol_features, fill_value=0.0)
        sig_input = window.iloc[[-2]].reindex(columns=sig_features, fill_value=0.0)

        try:
            prob = signal_model.final_model.predict_proba(sig_input)[0][1]
            if prob >= 0.85:
                prediction = "UP"
            elif prob <= 0.15:
                prediction = "DOWN"
            else:
                continue  # confidence too low

            signal_type = "xgboost_bullish" if prediction == "UP" else "xgboost_bearish"
            conf_str = f"Conf={prob:.2%}" if prediction == "UP" else f"Conf={(1 - prob):.2%}"
            signal_logger.add_signal(signal_type, now, window["close"].iloc[-2], conf_str)

            prediction_logger.record_prediction(
                timestamp=now,
                prediction=prediction,
                close_now=window['close'].iloc[-1],
                close_prev=window['close'].iloc[-2],
                confidence=prob
            )

        except Exception as e:
            print(f"[{now}] {regime} → ERROR during prediction: {e}")
            continue

    df_result = prediction_logger.to_dataframe()
    num_preds = len(df_result)
    hit_rate = df_result["hit"].mean() if num_preds > 0 else 0.0

    print(f"[{regime}] Finished with {num_preds} predictions, hit rate: {hit_rate:.2%}")

    if num_preds > 0:
        summary.append({
            "regime": regime,
            "hit_rate": round(hit_rate * 100, 2),
            "num_predictions": num_preds
        })
    else:
        print(f"[{regime}] No confident predictions made.")

# === Final Summary ===
print("\n=== Backtest Summary ===")
if summary:
    summary_df = pd.DataFrame(summary)
    summary_df = summary_df.sort_values(by="hit_rate", ascending=False).reset_index(drop=True)
    print(summary_df)
else:
    print("No valid predictions were made across all regimes.")



=== Evaluating Regime: HighVol_DownTrend ===
[HighVol_DownTrend] Rows after feature prep: 501
[HighVol_DownTrend] Finished with 68 predictions, hit rate: 39.71%

=== Evaluating Regime: HighVol_UpTrend ===
[HighVol_UpTrend] Rows after feature prep: 501
[HighVol_UpTrend] Finished with 42 predictions, hit rate: 40.48%

=== Evaluating Regime: Sideways ===
[Sideways] Rows after feature prep: 501
[Sideways] Finished with 132 predictions, hit rate: 31.06%

=== Evaluating Regime: LowVol_DownTrend ===
[LowVol_DownTrend] Rows after feature prep: 501
[LowVol_DownTrend] Finished with 61 predictions, hit rate: 59.02%

=== Evaluating Regime: LowVol_UpTrend ===
[LowVol_UpTrend] Rows after feature prep: 501
[LowVol_UpTrend] Finished with 117 predictions, hit rate: 50.43%

=== Backtest Summary ===
              regime  hit_rate  num_predictions
0   LowVol_DownTrend     59.02               61
1     LowVol_UpTrend     50.43              117
2    HighVol_UpTrend     40.48               42
3  HighVol_Down